In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import plotly.express as px
from prophet import Prophet

# Retail Transaction Dataset Overview

## **Key Characteristics**
- **Rows**: 541,909 (each row represents a product in a transaction)
- **Columns**: 8 (details about transactions, products, customers, and locations)
- **Time Period**: Includes timestamped transactions (`InvoiceDate`), enabling temporal analysis
- **Missing Data**:
  - `Description`: ~0.27% missing
  - `CustomerID`: ~25% missing
- **Diversity**:
  - **Products**: 4,070 unique (`StockCode`)
  - **Countries**: 38 markets
  - **Customers**: 4,372 distinct (`CustomerID`, excluding missing)

---

## **Column Descriptions**
| **Column**       | **Description**                                                                 |
|-------------------|---------------------------------------------------------------------------------|
| **InvoiceNo**     | Unique identifier for each transaction.                                         |
| **StockCode**     | Unique identifier for products.                                                |
| **Description**   | Product name. Missing values suggest some products lack descriptions.          |
| **Quantity**      | Number of units sold. Negative values likely indicate returns or adjustments.  |
| **InvoiceDate**   | Date and time of the transaction.                                               |
| **UnitPrice**     | Price per unit in the transaction's currency.                                   |
| **CustomerID**    | Unique identifier for customers (missing for anonymous buyers).                |
| **Country**       | Country where the customer resides.                                            |

---

In [15]:
df = pd.read_excel('Online Retail.xlsx')

In [16]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [17]:
df.shape

(541909, 8)

In [18]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [20]:
df.nunique()

,0
InvoiceNo,25900
StockCode,4070
Description,4223
Quantity,722
InvoiceDate,23260
UnitPrice,1630
CustomerID,4372
Country,38


In [21]:
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


# DataPreprocessor Class

The `DataPreprocessor` class is designed to clean and transform retail transaction data. It contains several methods for handling missing values, parsing columns, and creating new derived columns.

## **Class Overview**
The class is initialized with:
- **`raw_data`**: A copy of the input dataset (`DataFrame`) to avoid modifying the original dataset.
- **`categories`**: A list of categories that the data can be classified into. Example categories include 'Grocery', 'Home Decoration', 'Appliances', 'Bag', and 'Beauty Products'.


In [22]:
categories = [
    'Grocery', 'Home Decoration',
    'Appliances', 'Fashion', 'Beauty Products',
]

# DataPreprocessor Class

The `DataPreprocessor` class is designed to clean and transform retail transaction data. It contains several methods for handling missing values, parsing columns, and creating new derived columns.

## **Class Overview**
The class is initialized with:
- **`raw_data`**: A copy of the input dataset (`DataFrame`) to avoid modifying the original dataset.
- **`categories`**: A dictionary or additional parameter set for handling category-based processing (though not explicitly used in the provided methods).

---

## **Methods**

### **1. `parse_CustomerID()`**
- **Purpose**:
  - Fills missing `CustomerID` values with `0` and converts the `CustomerID` column to integers.
- **Returns**:
  - The updated `DataPreprocessor` instance with the `CustomerID` column processed.

### **2. `fill_missing_customer_id()`**
- **Purpose**:
  - Replaces missing `CustomerID` values (denoted by `0`) with unique 5-digit integers.
  - Ensures no duplicate `CustomerID` values.
- **Process**:
  - Finds the maximum current `CustomerID`, then assigns unique IDs to missing entries.
  - Prints the number of IDs changed and the newly assigned IDs.
- **Returns**:
  - The updated `DataPreprocessor` instance.

### **3. `parse_Description()`**
- **Purpose**:
  - Drops rows where the `Description` field is missing (`NaN`).
- **Returns**:
  - The cleaned dataset with no missing `Description` values.

### **4. `parse_Gender()`**
- **Purpose**:
  - Adds a new `Gender` column with the default value `'Unisex'` for all rows.
- **Returns**:
  - The updated dataset with the `Gender` column added.

### **5. `parse_UserType()`**
- **Purpose**:
  - Adds a new `UserType` column with the default value `'General'`.
- **Returns**:
  - The updated dataset with the `UserType` column added.

### **6. `parse_StockCode()`**
- **Purpose**:
  - Trims the `StockCode` to the first 5 characters.
- **Returns**:
  - The updated dataset with standardized `StockCode` values.

### **7. `parse_OrderType()`**
- **Purpose**:
  - Adds an `OrderType` column:
    - `'Delivered'` for normal orders.
    - `'Cancelled'` for orders with `InvoiceNo` starting with `'C'`.
- **Returns**:
  - The updated dataset with the `OrderType` column added.

### **8. `parse_Total()`**
- **Purpose**:
  - Adds a `Total` column, calculating the total price per transaction (`Quantity * UnitPrice`).
- **Returns**:
  - The updated dataset with the `Total` column added.

### **9. `parse_Day()`**
- **Purpose**:
  - Converts the `InvoiceDate` column to `datetime`.
  - Extracts the `DayOfWeek` (e.g., Monday, Tuesday, etc.) from `InvoiceDate`.
- **Returns**:
  - The updated dataset with the `DayOfWeek` column added.


In [23]:
class DataPreprocessor:
    def __init__(self, raw_data, categories):
        self.df = raw_data.copy()  # Work on a copy to avoid modifying the original DataFrame
        self.categories = categories

    def parse_CustomerID(self):
        self.df['CustomerID'] = self.df['CustomerID'].fillna(0).astype(int)
        return self

    def fill_missing_customer_id(self):
        """
        Fills missing CustomerID values with unique 5-digit numbers.

        Returns:
            DataPreprocessor: The updated DataPreprocessor instance.
        """
        max_customer_id = self.df['CustomerID'].max()
        next_customer_id = max_customer_id + 1
        changed_customer_ids = []
        count = 0

        for index, row in self.df.iterrows():
            if row['CustomerID'] == 0:  # Assuming missing values are already filled with 0
                while next_customer_id in self.df['CustomerID'].values:
                    next_customer_id += 1
                self.df.loc[index, 'CustomerID'] = next_customer_id
                changed_customer_ids.append(next_customer_id)
                next_customer_id += 1
                count += 1


        return self

    def parse_Description(self):
        self.df = self.df.dropna(subset=['Description']).copy()
        return self

    def parse_Gender(self):
      def assign_gender(description):
        # Convert description to lowercase for case-insensitive comparison
        description = str(description).lower()

        # Check for keywords related to each gender
        if any(keyword in description for keyword in ["men", "men's", "male", "gentlemen"]):
            return 'Men'
        elif any(keyword in description for keyword in ["women", "women's", "ladies", "female"]):
            return 'Women'
        elif any(keyword in description for keyword in ["kid", "kids", "children", "child", "boys", "girls"]):
            return 'Kids'
        else:
            return 'Unisex'

      # Apply the gender assignment function to each row in the 'Description' column
      self.df['Gender'] = self.df['Description'].apply(assign_gender)
      return self


    def parse_UserType(self):
      def identify_user_type(description):
        description = str(description).lower()
        if any(keyword in description for keyword in ["business", "office", "corporate", "professional"]):
            return 'Business'
        elif any(keyword in description for keyword in ["student", "school", "college", "education"]):
            return 'Student'
        elif any(keyword in description for keyword in ["gift", "present", "occasion", "celebration"]):
            return 'Gift Buyer'
        elif any(keyword in description for keyword in ["home", "household", "decor", "furniture"]):
            return 'Homeowner'
        else:
            return 'General'

    # Apply the identify_user_type function to the 'Description' column
      self.df['UserType'] = self.df['Description'].apply(identify_user_type)
      return self


    def parse_StockCode(self):
        self.df.loc[:, 'StockCode'] = self.df['StockCode'].astype(str).str[:5]
        return self

    def parse_OrderType(self):
        self.df.loc[:, 'OrderType'] = 'Delivered'
        self.df.loc[self.df['InvoiceNo'].astype(str).str.startswith('C'), 'OrderType'] = 'Cancelled'
        return self

    def parse_Total(self):
        self.df.loc[:, 'Total'] = self.df['Quantity'] * self.df['UnitPrice']
        return self

    def parse_Day(self):
        self.df.loc[:, 'InvoiceDate'] = pd.to_datetime(self.df['InvoiceDate'])
        self.df.loc[:, 'DayOfWeek'] = self.df['InvoiceDate'].dt.day_name()
        return self
    def parse_Category(self):
        # Handle missing and non-string values
        self.df['Description'] = self.df['Description'].fillna('').astype(str)

        # Create a TF-IDF vectorizer
        vectorizer = TfidfVectorizer(stop_words='english')

        # Fit and transform the 'Description' column
        tfidf_matrix = vectorizer.fit_transform(self.df['Description'])

        # Apply KMeans clustering
        num_clusters = len(self.categories)  # Number of clusters based on the provided categories
        kmeans = KMeans(n_clusters=num_clusters, random_state=0, n_init=10)
        kmeans.fit(tfidf_matrix)

        # Add cluster labels to the DataFrame
        self.df['Category'] = kmeans.labels_

        # Map cluster labels to category names (if available)
        if self.categories:
            category_mapping = {i: category for i, category in enumerate(self.categories)}
            self.df['Category'] = self.df['Category'].map(category_mapping)

        return self



# DataQuerier Class

The `DataQuerier` class is designed to perform filtering operations on processed retail transaction data. It allows you to query the data based on various attributes such as gender, order status, total amount, country, day of the week, and user type.

## **Class Overview**
The class is initialized with:
- **`filtered_data`**: A pre-processed dataset (usually after cleaning and transformations) that the class will use to perform queries.

---

## **Methods**

### **1. `by_gender(gender)`**
- **Purpose**:
  - Filters the dataset by a specific gender.
- **Arguments**:
  - `gender` (str): The gender to filter by (e.g., `'Men'`, `'Women'`, `'Unisex'`).
- **Returns**:
  - A filtered `DataFrame` containing only the rows that match the specified gender.

### **2. `by_cancellation()`**
- **Purpose**:
  - Filters the dataset to show only cancelled orders.
- **Returns**:
  - A filtered `DataFrame` containing only the cancelled orders (i.e., orders with `OrderType` as `'Cancelled'`).

### **3. `by_total_above(threshold)`**
- **Purpose**:
  - Filters the dataset to show orders with a total amount greater than a specified threshold.
- **Arguments**:
  - `threshold` (float): The minimum total amount for the filter.
- **Returns**:
  - A filtered `DataFrame` containing only orders where the total amount exceeds the specified threshold.

### **4. `by_country(country)`**
- **Purpose**:
  - Filters the dataset by a specific country.
- **Arguments**:
  - `country` (str): The country to filter by (e.g., `'United Kingdom'`).
- **Returns**:
  - A filtered `DataFrame` containing only the orders from the specified country.

### **5. `by_day_of_week(day)`**
- **Purpose**:
  - Filters the dataset by a specific day of the week.
- **Arguments**:
  - `day` (str): The day of the week to filter by (e.g., `'Monday'`, `'Tuesday'`).
- **Returns**:
  - A filtered `DataFrame` containing only the orders from the specified day of the week.

### **6. `by_user_type(user_type)`**
- **Purpose**:
  - Filters the dataset by user type.
- **Arguments**:
  - `user_type` (str): The user type to filter by (e.g., `'Business'`, `'Student'`).
- **Returns**:
  - A filtered `DataFrame` containing only the rows that match the specified user type.

---

In [24]:
class DataQuerier:
    def __init__(self, filtered_data):
        """
        Initializes the DataQuerier with filtered data.

        Args:
            filtered_data (pd.DataFrame): The processed data.
        """
        self.df = filtered_data

    def by_gender(self, gender):
        """
        Filters the data by a specific gender.

        Args:
            gender (str): The gender to filter by (e.g., 'Men', 'Women', 'Unisex').

        Returns:
            pd.DataFrame: Filtered data.
        """
        filtered = self.df[self.df['Gender'] == gender]
        print(f"Filtered by Gender = {gender}:\n{filtered.head()}")
        return filtered

    def by_cancellation(self):
        """
        Filters the data to show only cancelled orders.

        Returns:
            pd.DataFrame: Filtered data.
        """
        filtered = self.df[self.df['OrderType'] == 'Cancelled']
        print(f"Filtered Cancelled Orders:\n{filtered.head()}")
        return filtered

    def by_total_above(self, threshold):
        """
        Filters the data to show orders with a total above a specific threshold.

        Args:
            threshold (float): The minimum total value.

        Returns:
            pd.DataFrame: Filtered data.
        """
        filtered = self.df[self.df['Total'] > threshold]
        print(f"Filtered by Total > {threshold}:\n{filtered.head()}")
        return filtered

    def by_country(self, country):
        """
        Filters the data by country.

        Args:
            country (str): The country to filter by.

        Returns:
            pd.DataFrame: Filtered data.
        """
        filtered = self.df[self.df['Country'] == country]
        print(f"Filtered by Country = {country}:\n{filtered.head()}")
        return filtered

    def by_day_of_week(self, day):
        """
        Filters the data by a specific day of the week.

        Args:
            day (str): The day of the week (e.g., 'Monday', 'Tuesday').

        Returns:
            pd.DataFrame: Filtered data.
        """
        filtered = self.df[self.df['DayOfWeek'] == day]
        print(f"Filtered by DayOfWeek = {day}:\n{filtered.head()}")
        return filtered

    def by_user_type(self, user_type):
        """
        Filters the data by user type.

        Args:
            user_type (str): The user type to filter by (e.g., 'Business', 'Student').

        Returns:
            pd.DataFrame: Filtered data.
        """
        filtered = self.df[self.df['UserType'] == user_type]
        print(f"Filtered by UserType = {user_type}:\n{filtered.head()}")
        return filtered


# ConsumerTransactionDB Class

The `ConsumerTransactionDB` class is designed to handle the preprocessing of consumer transaction data, apply transformations, and perform queries on the processed data. It leverages the `DataPreprocessor` for cleaning and transforming the data, and the `DataQuerier` for executing queries on the processed dataset.

## **Class Overview**
The class is initialized with:
- **`raw_data`**: The raw input data (typically a pandas `DataFrame`).
- **`categories`**: A list of predefined categories to be used for clustering the descriptions of the products.

The `ConsumerTransactionDB` class encapsulates both preprocessing and querying functionalities, making it easier to process and retrieve insights from the data.

---

## **Methods**

### **1. `__init__(self, raw_data, categories)`**
- **Purpose**:
  - Initializes the `ConsumerTransactionDB` instance with raw data and categories.
- **Arguments**:
  - `raw_data` (pd.DataFrame): The raw input dataset.
  - `categories` (list): A list of predefined categories for description clustering.

### **2. `preprocess(self)`**
- **Purpose**:
  - Preprocesses the raw data using the `DataPreprocessor` class. This method applies various transformations and cleaning operations to prepare the data for further analysis.
- **Returns**:
  - The `ConsumerTransactionDB` instance with processed data and a `DataQuerier` instance initialized for querying.

### **3. `query(self, method_name, *args, **kwargs)`**
- **Purpose**:
  - Executes a query method from the `DataQuerier` class on the processed data.
- **Arguments**:
  - `method_name` (str): The name of the query method to be invoked (e.g., `'by_gender'`, `'by_total_above'`).
  - `*args`: Positional arguments for the query method.
  - `**kwargs`: Keyword arguments for the query method.
- **Returns**:
  - The result of the query method, which is a filtered `pd.DataFrame`.
- **Raises**:
  - `ValueError` if data has not been preprocessed.
  - `AttributeError` if the specified query method does not exist in `DataQuerier`.

### **4. `get_processed_data(self)`**
- **Purpose**:
  - Returns the processed data after preprocessing.
- **Returns**:
  - The processed `pd.DataFrame`.
- **Raises**:
  - `ValueError` if the data has not been processed yet.

In [25]:
class ConsumerTransactionDB:
    def __init__(self, raw_data, categories):
        """
        Initializes the ConsumerTransactionDB with raw data and predefined categories.

        Args:
            raw_data (pd.DataFrame): The raw input data.
            categories (list): List of predefined categories for description clustering.
        """
        self.raw_data = raw_data
        self.categories = categories
        self.preprocessor = DataPreprocessor(self.raw_data, self.categories)
        self.processed_data = None
        self.querier = None

    def preprocess(self):
        """
        Preprocesses the raw data using the DataPreprocessor.
        """
        self.processed_data = (
            self.preprocessor
                .parse_Description()
                .parse_CustomerID()
                .fill_missing_customer_id()
                .parse_Gender()
                .parse_Category()
                .parse_UserType()
                .parse_StockCode()
                .parse_OrderType()
                .parse_Total()
                .parse_Day()
                .df
        )
        self.querier = DataQuerier(self.processed_data)  # Initialize DataQuerier with processed data
        print("Data preprocessing complete.")
        return self

    def query(self, method_name, *args, **kwargs):
        """
        Performs a query on the processed data using the DataQuerier.

        Args:
            method_name (str): The name of the query method to invoke.
            *args: Positional arguments for the query method.
            **kwargs: Keyword arguments for the query method.

        Returns:
            pd.DataFrame: The result of the query.
        """
        if not self.querier:
            raise ValueError("Data must be preprocessed before querying.")
        if not hasattr(self.querier, method_name):
            raise AttributeError(f"Query method '{method_name}' does not exist.")
        return getattr(self.querier, method_name)(*args, **kwargs)

    def get_processed_data(self):
        """
        Returns the processed data.

        Returns:
            pd.DataFrame: The processed DataFrame.
        """
        if self.processed_data is None:
            raise ValueError("Data has not been processed yet.")
        return self.processed_data


In [26]:
# Assuming `df` is your raw data and `categories` is a predefined list of categories.
categories = ['Grocery', 'Home Decoration', 'Appliances', 'Bag', 'Beauty Products']

# Initialize ConsumerTransactionDB
transaction_db = ConsumerTransactionDB(df, categories)

# Preprocess the data
transaction_db.preprocess()

# Access processed data
processed_data = transaction_db.get_processed_data()
print(processed_data.head())

# Perform queries
gender_filtered = transaction_db.query('by_gender', 'Unisex')
print(gender_filtered.head())

cancelled_orders = transaction_db.query('by_cancellation')
print(cancelled_orders.head())

high_value_orders = transaction_db.query('by_total_above', 50)
print(high_value_orders.head())

uk_orders = transaction_db.query('by_country', 'United Kingdom')
print(uk_orders.head())

monday_orders = transaction_db.query('by_day_of_week', 'Monday')
print(monday_orders.head())

business_users = transaction_db.query('by_user_type', 'Business')
print(business_users.head())


Data preprocessing complete.
  InvoiceNo StockCode                          Description  Quantity  \
0    536365     85123   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365     84406       CREAM CUPID HEARTS COAT HANGER         8   
3    536365     84029  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365     84029       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  Gender  \
0 2010-12-01 08:26:00       2.55       17850  United Kingdom  Unisex   
1 2010-12-01 08:26:00       3.39       17850  United Kingdom  Unisex   
2 2010-12-01 08:26:00       2.75       17850  United Kingdom  Unisex   
3 2010-12-01 08:26:00       3.39       17850  United Kingdom  Unisex   
4 2010-12-01 08:26:00       3.39       17850  United Kingdom  Unisex   

          Category UserType  OrderType  Total  DayOfWeek  
0  Beauty Products  General  Delivered  15.30 

In [27]:
business_users = transaction_db.query('by_user_type', 'Business')
print(business_users.head())

Filtered by UserType = Business:
     InvoiceNo StockCode                      Description  Quantity  \
863     536464     22312       OFFICE MUG WARMER POLKADOT         1   
1331    536535     22311  OFFICE MUG WARMER BLACK+SILVER          6   
1550    536544     22314      OFFICE MUG WARMER CHOC+BLUE         1   
2153    536562     22313           OFFICE MUG WARMER PINK         6   
2154    536562     22314      OFFICE MUG WARMER CHOC+BLUE         6   

             InvoiceDate  UnitPrice  CustomerID         Country  Gender  \
863  2010-12-01 12:23:00       2.95       17968  United Kingdom  Unisex   
1331 2010-12-01 13:38:00       2.95       15605  United Kingdom  Unisex   
1550 2010-12-01 14:32:00       5.91       18395  United Kingdom  Unisex   
2153 2010-12-01 15:08:00       2.95       13468  United Kingdom  Unisex   
2154 2010-12-01 15:08:00       2.95       13468  United Kingdom  Unisex   

     Category  UserType  OrderType  Total  DayOfWeek  
863   Grocery  Business  Delivered

In [28]:

processed_data.head(4)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Gender,Category,UserType,OrderType,Total,DayOfWeek
0,536365,85123,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,Unisex,Beauty Products,General,Delivered,15.30,Wednesday
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Unisex,Bag,General,Delivered,20.34,Wednesday
2,536365,84406,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,Unisex,Grocery,General,Delivered,22.00,Wednesday
3,536365,84029,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Unisex,Grocery,General,Delivered,20.34,Wednesday


# Conert to csv file

In [29]:


processed_data.to_csv('processed_data.csv', index=False)

# Sales data for the last 10 days:

In [30]:
#n_days = int(input("Enter the number of days: "))
n_days = 10

try:
    processed_data = pd.read_csv('processed_data.csv', parse_dates=['InvoiceDate'])
except FileNotFoundError:
    print("Error: 'processed_data.csv' not found. Please run the previous code blocks first.")
    # Handle the error appropriately, e.g., exit the script or provide a default DataFrame.
    exit()


# Convert 'InvoiceDate' to datetime if it's not already
if not pd.api.types.is_datetime64_any_dtype(processed_data['InvoiceDate']):
    processed_data['InvoiceDate'] = pd.to_datetime(processed_data['InvoiceDate'])


# Calculate the date n days ago
n_days_ago = processed_data['InvoiceDate'].max() - pd.DateOffset(days=n_days)

# Filter data for the last n days
last_n_days_sales = processed_data[processed_data['InvoiceDate'] >= n_days_ago]


print(f"Sales data for the last {n_days} days:")
last_n_days_sales

Sales data for the last 10 days:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Gender,Category,UserType,OrderType,Total,DayOfWeek
508547,579413,23406,HOME SWEET HOME KEY HOLDER,2,2011-11-29 12:50:00,6.25,13668,United Kingdom,Unisex,Grocery,Homeowner,Delivered,12.50,Tuesday
508548,579413,22573,STAR WOODEN CHRISTMAS DECORATION,6,2011-11-29 12:50:00,0.85,13668,United Kingdom,Unisex,Grocery,Homeowner,Delivered,5.10,Tuesday
508549,579413,22578,WOODEN STAR CHRISTMAS SCANDINAVIAN,10,2011-11-29 12:50:00,0.29,13668,United Kingdom,Unisex,Grocery,General,Delivered,2.90,Tuesday
508550,579413,22577,WOODEN HEART CHRISTMAS SCANDINAVIAN,10,2011-11-29 12:50:00,0.29,13668,United Kingdom,Unisex,Grocery,General,Delivered,2.90,Tuesday
508551,579413,22574,HEART WOODEN CHRISTMAS DECORATION,6,2011-11-29 12:50:00,0.85,13668,United Kingdom,Unisex,Grocery,Homeowner,Delivered,5.10,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540450,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,Unisex,Grocery,General,Delivered,10.20,Friday
540451,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,Kids,Grocery,General,Delivered,12.60,Friday
540452,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,Kids,Grocery,General,Delivered,16.60,Friday
540453,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,Kids,Grocery,General,Delivered,16.60,Friday


# Total revenue Generate in the last 5 days

In [31]:
#n_days= int(input("Enter the number of days: "))
n_days= 5


# Calculate total revenue for the last 10 days
total_revenue = last_n_days_sales['Total'].sum()
print(f"Total revenue for the last {n_days} days: {total_revenue}")

Total revenue for the last 5 days: 532379.51


# Sales Revenue by Country

### Top N Country by Total Sales

| Rank | Country           | Total Revenue     |
|------|-------------------|-------------------|
| 1    | United Kingdom    | 8,187,806.364     |
| 2    | Netherlands       | 284,661.540       |
| 3    | EIRE              | 263,276.820       |
| 4    | Germany           | 221,698.210       |
| 5    | France            | 197,403.900       |

### Key Insights:
1. **Top Performer:** The United Kingdom dominates with a significant revenue share of over 8.1 million.
2. **Second Place:** The Netherlands follows with 284k, a notable gap from the leader.
3. **Close Competition:** EIRE, Germany, and France show similar revenue levels but are far behind the United Kingdom.


In [32]:


def top_n_country_wise_sales(n):
    """
    Displays the top N countries by total sales and generates a bar chart.

    Args:
        n (int): The number of top countries to display.
    """
    try:
        # Load the processed data
        processed_data = pd.read_csv('processed_data.csv')
    except FileNotFoundError:
        print("Error: 'processed_data.csv' not found. Please run the previous code blocks first.")
        return

    # Calculate total sales per country
    country_sales = processed_data.groupby('Country')['Total'].sum().reset_index()

    # Sort by total sales in descending order
    country_sales = country_sales.sort_values(by='Total', ascending=False)

    # Get the top N countries
    top_n_countries = country_sales.head(n)

    # Display the top N countries and their sales
    print(f"Top {n} Countries by Total Sales:")
    print(top_n_countries)

    # Create a bar chart
    fig = px.bar(top_n_countries, x='Country', y='Total',
                 title=f'Top {n} Countries by Total Sales',
                 labels={'Total': 'Total Sales', 'Country': 'Country'})
    fig.show()

n_country = int(input("Enter the number of top countries to display: "))
top_n_country_wise_sales(n_country)  # Shows the top 5 countries

Enter the number of top countries to display: 5
Top 5 Countries by Total Sales:
           Country        Total
36  United Kingdom  8187806.364
24     Netherlands   284661.540
10            EIRE   263276.820
14         Germany   221698.210
13          France   197403.900


In [33]:
def plot_gender_purchase_frequency(df, top_n_countries):
    """
    Plots the purchase frequency by gender for the top N countries.

    Args:
        df (pd.DataFrame): The input DataFrame.
        top_n_countries (int): The number of top countries to consider.
    """
    try:
        # Load the processed data
        processed_data = pd.read_csv('processed_data.csv')
    except FileNotFoundError:
        print("Error: 'processed_data.csv' not found. Please run the previous code blocks first.")
        return

    # Calculate total sales per country
    country_sales = processed_data.groupby('Country')['Total'].sum().reset_index()

    # Sort by total sales in descending order
    country_sales = country_sales.sort_values(by='Total', ascending=False)

    # Get the top N countries
    top_countries = country_sales.head(top_n_countries)['Country'].tolist()

    # Filter data for the top countries
    top_country_data = processed_data[processed_data['Country'].isin(top_countries)]

    # Group data by country and gender, then count purchases
    gender_purchase_counts = top_country_data.groupby(['Country', 'Gender'])['InvoiceNo'].count().reset_index()

    # Create the plot
    fig = px.bar(gender_purchase_counts, x='Country', y='InvoiceNo', color='Gender',
                 title=f'Purchase Frequency by Gender for Top {top_n_countries} Countries',
                 labels={'InvoiceNo': 'Number of Purchases', 'Country': 'Country', 'Gender': 'Gender'})
    fig.show()

# Example usage (replace with your desired number of top countries)
n_top_countries = int(input("Enter the number of top countries: "))
plot_gender_purchase_frequency(processed_data, n_top_countries)

Enter the number of top countries: 5


In [34]:
def plot_gender_purchase_frequency_last_n_countries(df, n_countries):

    # Calculate total sales per country
    country_sales = processed_data.groupby('Country')['Total'].sum().reset_index()

    # Sort by total sales in descending order
    country_sales = country_sales.sort_values(by='Total', ascending=False)

    # Get the last N countries
    last_n_countries = country_sales.tail(n_countries)['Country'].tolist()

    # Filter data for the last N countries
    last_n_country_data = processed_data[processed_data['Country'].isin(last_n_countries)]

    # Group data by country and gender, then count purchases
    gender_purchase_counts = last_n_country_data.groupby(['Country', 'Gender'])['InvoiceNo'].count().reset_index()

    # Create the plot
    fig = px.bar(gender_purchase_counts, x='Country', y='InvoiceNo', color='Gender',
                 title=f'Purchase Frequency by Gender for Last {n_countries} Countries',
                 labels={'InvoiceNo': 'Number of Purchases', 'Country': 'Country', 'Gender': 'Gender'})
    fig.show()

# Example usage (replace with your desired number of last countries)
n_last_countries = int(input("Enter the number of last countries: "))
plot_gender_purchase_frequency_last_n_countries(processed_data, n_last_countries)

Enter the number of last countries: 5


# Distribution by Gender

In [35]:
def plot_gender_donut(df):
    """Plots a donut chart showing the purchase distribution by gender."""
    try:
        gender_counts = df['Gender'].value_counts()
    except KeyError:
        print("Error: 'Gender' column not found in the DataFrame.")
        return

    fig = px.pie(gender_counts, values=gender_counts.values, names=gender_counts.index, hole=0.3,
                 title="Purchase Distribution by Gender")
    fig.show()


plot_gender_donut(processed_data)


In [36]:
def plot_user_type_distribution(df):
    """Plots a donut chart showing the purchase distribution by UserType."""
    try:
        user_type_counts = df['UserType'].value_counts()
    except KeyError:
        print("Error: 'UserType' column not found in the DataFrame.")
        return

    fig = px.pie(user_type_counts, values=user_type_counts.values, names=user_type_counts.index, hole=0.3,
                 title="Purchase Distribution by User Type")
    fig.show()

plot_user_type_distribution(processed_data)

# Total Sales by Day of the Week


In [37]:
# Group data by DayOfWeek and sum the Total for each day
dayofweek_sales = processed_data.groupby('DayOfWeek')['Total'].sum().reset_index()


# Create the plot with improved UI/UX
fig = px.bar(
    dayofweek_sales,
    x='DayOfWeek',
    y='Total',
    title='Total Sales by Day of the Week',
    labels={'Total': 'Total Sales ($)', 'DayOfWeek': 'Day of the Week'},
    color='Total',
    color_continuous_scale=px.colors.sequential.Blues,
    text='Total'  # Display totals on bars
)

# Customize layout for better readability
fig.update_layout(
    title=dict(x=0.5, font=dict(size=20, family='Arial')),
    xaxis=dict(title='Day of the Week', tickfont=dict(size=12)),
    yaxis=dict(title='Total Sales', tickfont=dict(size=12)),

    font=dict(family="Arial", size=14),
)

# Add text to bars
fig.update_traces(
    texttemplate='%{text:.2s}', textposition='outside', textfont=dict(size=12)
)

# Show the plot
fig.show()

#Total Sales over Time

* Highest Price : $168469.60

* Lowest Price : -$168469.60

In [38]:
# Group data by InvoiceDate and sum Total
invoice_date_sales = processed_data.groupby('InvoiceDate')['Total'].sum().reset_index()

fig = px.line(
    invoice_date_sales,
    x='InvoiceDate',
    y='Total',
    title='Total Sales over Time',
    labels={'InvoiceDate': 'Invoice Date', 'Total': 'Total Sales ($)'},
    markers=True,  # Add markers to emphasize data points
    line_shape='spline',  # Smooth line shape for aesthetics
    color_discrete_sequence=['#007988']  # Set line color to match branding
)

# Add a range slider for zoom functionality
fig.update_xaxes(
    rangeslider_visible=True,
    title_font=dict(size=14),
    tickangle=45,  # Angle ticks for better readability
    showgrid=True,  # Display gridlines for clarity
    gridcolor='lightgrey'
)

# Update Y-axis for better visibility
fig.update_yaxes(
    title_font=dict(size=14),
    showgrid=True,
    gridcolor='lightgrey'
)

# Add annotations for highest and lowest sales days
highest_sales = invoice_date_sales.loc[invoice_date_sales['Total'].idxmax()]
lowest_sales = invoice_date_sales.loc[invoice_date_sales['Total'].idxmin()]

fig.add_annotation(
    x=highest_sales['InvoiceDate'],
    y=highest_sales['Total'],
    text=f"Highest Sales: ${highest_sales['Total']:.2f}",
    showarrow=True,
    arrowhead=2,
    ax=-50,
    ay=-50,
    bgcolor="lightgreen"
)

fig.add_annotation(
    x=lowest_sales['InvoiceDate'],
    y=lowest_sales['Total'],
    text=f"Lowest Sales: ${lowest_sales['Total']:.2f}",
    showarrow=True,
    arrowhead=2,
    ax=50,
    ay=50,
    bgcolor="lightcoral"
)

# Update layout for better UI
fig.update_layout(
    title=dict(
        text='Total Sales Over Time<br><sub>Zoom in and out using the slider below</sub>',
        x=0.5,
        font=dict(size=20, family='Arial')
    ),
    plot_bgcolor='rgba(0,0,0,0)',  # Transparent plot background
    paper_bgcolor='rgba(255,255,255,1)',  # White paper background
    font=dict(family='Arial', size=14),
)

# Show the plot
fig.show()

# Sales Trends Over Time

In [41]:


# Example usage of the new function:
def plot_sales_trends(df, column_name):
    """
    Plots sales trends over time for a specified column.

    Args:
      df: Pandas DataFrame.
      column_name: The name of the column containing sales data to plot.
    """
    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in DataFrame.")
        return

    # Group data by 'InvoiceDate' and sum the specified column
    sales_trends = df.groupby('InvoiceDate')[column_name].sum().reset_index()

    fig = px.line(sales_trends, x='InvoiceDate', y=column_name,
                  title=f'{column_name} Trends Over Time',
                  labels={'InvoiceDate': 'Invoice Date', column_name: f'{column_name} Trends'},
                  markers=True,
                  line_shape='spline')

    fig.update_xaxes(rangeslider_visible=True)
    fig.show()

# Plot total sales trends
plot_sales_trends(processed_data, 'Total')

# Sales Data and Prediction

In [49]:
# Convert 'InvoiceDate' to datetime if it's not already
if not pd.api.types.is_datetime64_any_dtype(processed_data['InvoiceDate']):
    processed_data['InvoiceDate'] = pd.to_datetime(processed_data['InvoiceDate'])

# Prepare data for Prophet
sales_data = processed_data.groupby('InvoiceDate')['Total'].sum().reset_index()
sales_data.columns = ['ds', 'y']

# Initialize and fit the Prophet model
model = Prophet()
model.fit(sales_data)

# Create future dataframe for 10 days
future = model.make_future_dataframe(periods=10)

# Make predictions
forecast = model.predict(future)

# Plot sales data and prediction with different colors
fig = px.line(x=sales_data['ds'], y=sales_data['y'], title='Sales Data and Prediction', labels={'x':'Date', 'y':'Sales'}, color_discrete_sequence=['blue'])
fig.add_scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Prediction', line=dict(color='red'))
fig.show()

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsvxq9os6/3mb5mfmx.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsvxq9os6/lp4ij1oy.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=68540', 'data', 'file=/tmp/tmpsvxq9os6/3mb5mfmx.json', 'init=/tmp/tmpsvxq9os6/lp4ij1oy.json', 'output', 'file=/tmp/tmpsvxq9os6/prophet_modelo_asa9uv/prophet_model-20241123144730.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:47:30 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:47:33 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [56]:
# Calculate total sales for the last 10 days
last_10_days_sales = last_n_days_sales['Total'].sum()

# Calculate the number of new customers in the last 10 days
new_customers_last_10_days = len(last_n_days_sales['CustomerID'].unique())

print(f"Projected sales for the next 10 days: {forecast['yhat'].iloc[-10:].sum()}")
print(f"Number of new customers in the last 10 days: {new_customers_last_10_days}")
print(f"Total sales for the last 10 days: {last_10_days_sales}")

Projected sales for the next 10 days: 4412.9113542443865
Number of new customers in the last 10 days: 11158
Total sales for the last 10 days: 532379.51


In [39]:
processed_data.head(4)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Gender,Category,UserType,OrderType,Total,DayOfWeek
0,536365,85123,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,Unisex,Beauty Products,General,Delivered,15.30,Wednesday
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Unisex,Bag,General,Delivered,20.34,Wednesday
2,536365,84406,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,Unisex,Grocery,General,Delivered,22.00,Wednesday
3,536365,84029,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Unisex,Grocery,General,Delivered,20.34,Wednesday
